# Thompson Sampling

Thompson Sampling has been show, academically, to outperform all other approaches on the contextual-bandit problem.  The approach does so by performing exploration over which it has less confidence, while being fairly greedy over parameters with high confidence.

<img src="thompsonsamplingdescwiki.png">

where bayes rule says:

So, Thompson sampling requires a predictive model that computes the posterior distribution over the model parameters given the data, so that it may sample from this distribution at each action step.

As per bayes rule the posterior distribution over model parameters is:

<img src="bayes_rule_proportional.png">

The three components of this equation are, the posterior distribution on the left of the equation, the likelihood function in the center, and the prior on the right.

Although this looks simple, what makes this hard for most algorithms is that even though you can easily compute P(theta_i | Data), the posterior probability of a particular parameter vector, finding the best theta_i would require knowing the whole P(theta | Data).  

Many algorithms, including the LR versions we currently are testing, approximate solving ArgMax_i(P(theta_i | Data)) with iterative, gradient based approaches.  These approaches offer computational efficiency, but do not provide a full description of P(theta | Data).  At most they provide the optimization history for each training session.

# Logistic Regression

We've been playing around with LR as a main algorithm for awhile now.

The main benefits of Logistic Regression for us is:

- It produces meaningfull probabilities of outcomes P( click | campaign, request ) e [0,1]
- It's a simple algorithm (simple, lightweight set of parameters)
- Learning is simple and efficient (for local optima)
- Prediction, based on learned parameters, is easy to impliment 

The tools we're using for LR are optimizing logloss, in lieu of P(theta | Data).  Which is a fine proxy for iterative methods.

For a bayesian model we'd prefer to learn the posterior distribution over the model parameters.  Typical bayesian linear regression can be easily modeled with a gaussian-gamma model, where the posterior distribution over the model parameters is has a known shape, the gamma distribution.  Logistic Regression, a Generalized Linear Model, models P( y_i | x_i ) = logit(x_i.dot(theta)), where typical linear regression is passes through the logit function mapping (-inf, inf) to [0,1].  If we were to try to compute the posterior distribution over the model parameters, it would go something like this.

Logistic function:
<img src="logisticFunction.png">

Predictive probability:
<img src="logisticRegessionPredictiveProb.png">

Posterior distribution over the model parameters:
<img src="logisticRegressionPosteriorDist.png">

Again, bayes formula is made up of three parts.  The poserior on the left, the likelihood on the right in this case, and the prior in the middle.

If you take the logarithm of this you get.

Log-posterior:
<img src="logisticRegressionLogPosterior.png">

Now, this looks pretty managable computationally.  We have a few different pieces here, that require some matrix multiplication, but then you just add them all together.

- The C in the front is an itegration-constant that we can just ignore if we normalize the function to a distribution.
- The Pi_u(theta) part is the prior probability of theta_i, the probability of a single weight vector, under the assumptions we make about what are likely weight vectors (i.e. regularization, or previously learned models).
- Then the likelihood for the positive examples.
- Lastly, the likelihood for the negative examples.

So, like above, we have an equation to evaluate the P(theta_i | Data ) ( or log_P(theta_i | Data ) ).  

But what about finding a closed for solution for the whole distribution? 

The reason bayesian models are hard for Logistic Regression is, there isn't a [conjugate prior](https://en.wikipedia.org/wiki/Conjugate_prior) for the [Logit-Normal distribution](https://en.wikipedia.org/wiki/Logit-normal_distribution).  We know the likelihood of the model has the Logit-Normal shape because we're using categorical variables, and we know are LR parameters have normal-ish distributions.  It's a property of normal distributions, if they are independant, that the sum of several normal distributions is also a normal distribution.  So, the margin or x_i.dot(theta) is normally distributed, and then passed through a logistic function.  Making P(Y | X, theta) ~ Logit-Normal.  

In ortherwords, this is an intractable function in that, it can't be easily modeled with known distributions, that have easily computable parameters.

# Monte Carlo and Particle Filtering


For a long time, the way of extimating intractable functions was to generate samples in their range, then pass them through the function.  This process is called [monte carlo estimation](https://en.wikipedia.org/wiki/Monte_Carlo_method).  There are lots of flavors of monte carlo, but they all basically are trying to evaluate a function, over it's range, with some tractable number of samples.  A fast, and often used method for online-learning is Sample-Importance-Resampling, or particle filtering.

Particle filtering is online-learning approach to estimating a posterior distribution.  It uses a finite set of "particles" or samples within the range of the posterior distribution, and assigns an "importance" weight to them by evaluating the particle's value through the posterior distribution function.  An approximation to the Posterior distribution is then computed by normalizing the particles importance weights.  The online-learning part of the model involves using some new data to update the posterior distribution function, and re-generating particles using a sampling strategy utilizing the current generation of particles and their importances.

This tool is uses a lot in localization and navigation, and some great, animated examples exist online ([like here](https://www.youtube.com/watch?v=aUkBa1zMKv4)).

### In the localization context the elements of particles sampling are:
Posterior distribution: The distribution over current location, given history and current data.
Particles: Possible locations in space
Particle Importance: probability (or log-probability) of being at a particular location, given history and current data.
New data: Sensor information (Lidar, radar, gps, cameras and so on)


### In the context of a Probability model such as Logistic Regression:
Posterior distribution: The distribution over model parameters (for LR the model weights and intercept)
Particles: A particular weight vector
Particle Importance: Probability (or log-probability) of a particular weight vector, given data
New data: Predictions made, and the actual outcomes (for ad-serving, this is impressions serverd)

## Particle Filtering still suffers from the curse of dimensionality

So the question araises, how many particles does one need to do this proceedure?  Well, in localization, a hundred or so samples should be enough to estimate location in a limited size, 2D space.  What about 3D space?  Maybe 10,000 particles would get you about the same coverage of 3D space that 100 particles would get you in 2D space.  Well, what about 4D space, or 5D space?  The answer is "a lot more" particles.  Well, what if you're predictive model has 1 million dimensions?  The answer is "too many" particles; it's just not do-able.

Well, lets put that idea on hold and talk about something else.

# Ensambles and Bagging



Bagging (Bootstrap aggregation) is a very popular machine learning proceedure used to reduce model variance, and improve performance.  It training k models, from k datasets of size m, each created by sampling, with replacement, the original dataset of size n, where m < n.  The k models then each make their own predictions on new data, and their predictions are combined to produce a meta-prodiction for the whole meta-model.  Often this is combined with boosting, and other sampling rules to make each of the sub-models more havily biased.  In practice, a collection of more biased sub-models, leads to a lower variance overall model.

The bootstrap sampling proceedure has been around in the field of statistics for much longer than it has been in machine learning.  It was typically used to estimate confidence bunds for statistics when one is short on data.

# Ensables as particles

WIP.

Further outline

- Weak learners in an ensamble are each a hypothesis vectors in parameter (theta) space.
- Generating a bunch of weak learners is equivilant to generating particles in theta space.
- Since it is expensive to generate enough random particles to represent a complex theta distribution, we can find candidate thetas by training P weak learners, in a common theta space.  Each weak learner will attempt to find it's own optimal value of theta, ensuring we have better representation of higher likelihood regions of P(theta|data) (the posterior of theta).
- We can tune the spread of the ensamble particles (weak learners) with typical ensamble hyper-parameters, such as training on sub-portions of training data, weighting training examples, differing stopping criteria (maximum iterations or tolerance), differing gradient descent hyper-parameters, and so on.

- We score the particles using the overall posterior theta distribution.
- At prediction time, we can perform Thompson-like sampling by selecting M << P of the weak learners, without replacement.
- We can either select them in proportion to their log-likelihood score or we can select them uniformly, and apply their scores as weights towards the overall-prediction
- Additionally, we can 

# An Example

In [6]:
#%matplotlib inline
%reload_ext autoreload
import numpy as np
#import matplotlib.pyplot as plt
import math, sys, os
from numpy.random import randn
from sklearn.datasets import make_blobs
import json

# setup pyspark for IPython_notebooks
os.environ['SPARK_HOME'] = "/Users/robert.coleman/spark"
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))
shellFilePath = os.path.join(spark_home, 'python/pyspark/shell.py')
with open(shellFilePath) as f:
    exec(compile(f.read(), shellFilePath, 'exec'))

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by <module> at <ipython-input-2-97e7cd2d84d4>:17 